In [ ]:
labels, vectors = vsa.sample(5)
emp = vsa.empty(2, 10)
emp, vectors

In [ ]:
vsa.bind(vectors[0], vectors[1]), vsa.bundle(vectors[0], vectors[1])

In [ ]:
vsa.multibind(vectors), vsa.multiset(vectors, False)

In [ ]:
vsa.similarity(vectors[0], vectors[1])

In [ ]:
import torch
weights = torch.tensor([1,2,3,4,-1])
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))
# weights * vectors.transpose(-2,-1)

In [ ]:
weights = torch.tensor([[1,2,3,4,-1], [-1,-2,-3, -4, 1]])
vectors.unsqueeze(0).repeat(2,1,1)
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))

In [3]:
from main import *
from vsa.vsa import VSA

Test Passed: Label = [(1, 2, 0, 1)], Outcome = (1, 2, 0, 1)    Convergence: 1
